In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [2]:
#geopy installation

!pip install geopy
import geopy 

from geopy.geocoders import Nominatim
import time
from pprint import pprint

app = Nominatim(user_agent="tutorial")

In [3]:
url = "https://www.kollegierneskontor.dk/default.aspx?func=kkikportal.kollegiumlist&mid=40&topmenuid=34&lang=GB"
results = requests.get(url)

In [4]:
soup = BeautifulSoup(results.text, "html.parser")


In [5]:

#location = app.geocode("Dahlerupsgade 1 , 1603 København V, Denmark").raw
#print(location)


In [8]:

kkik = []

kollegium = soup.find_all('div', class_='kollegium row')


for container in kollegium:
    dorm_name = container.h3.a.text
    dorm_address = container.div.find('div', class_="address").text.strip().replace("\r\n               ", "").replace("\xa0", "")

    body = container.find('div', class_="body col-xs-12")
    rows = body.find_all("div", class_= "row")
    rows.pop(0)
    rows.pop(0)
    rows.pop(0)

    rooms_export = []

    for room in rows:
        room_details = room.find_all("div")

        roomtype = room_details[0].get_text().strip()
        size = room_details[1].get_text().strip()
        price = room_details[3].get_text().strip()

        rooms_export.append({
            "roomType":roomtype,
            "size":size,
            "price":price
        })

    kkik.append({
        "name":dorm_name,
        "address":dorm_address,
        "coordinates":coordinates(dorm_address),
        "rooms":rooms_export
    })


#Longitude and latitude

def coordinates(dorm_address):
    def get_location_by_address(address):
        time.sleep(1)
        try:
             return app.geocode(address).raw
        except:
            return {"latitude":"NaN", "longitude":"NaN"}
    address = dorm_address
    location = get_location_by_address(address)
    latitude = location["lat"]
    longitude = location["lon"]
    return [latitude, longitude]

#print(f"{latitude}, {longitude}")
# print all returned data
#pprint(location)

print(coordinates("Dalslandsgade 8, Copenhagen S"))





KeyboardInterrupt: 

In [42]:
import json

with open('json_kkik.json', 'w') as f:
    json.dump(kkik, f)
